Pipe, validacao e treinamento

In [110]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import joblib
import mlflow
import os

Carregar dataset csv limpo

In [111]:
caminho = "../data/tweets_limpo.csv"

if not os.path.exists(caminho):
    raise FileNotFoundError(f"Arquivo '{caminho}' não encontrado. Verifique o caminho.")

df = pd.read_csv(caminho)
display(df.head())

text     label  text_lenght
0            O produto e bom.  positivo           16
1           O produto e ruim.  negativo           17
2             Entrega rapida.  positivo           15
3  Produto chegou danificado.  negativo           26
4                  Muito bom!  positivo           10

Validacoes

In [112]:
assert df["text"].notnull().all(), "Coluna 'text' possui valores nulos."
assert df["label"].notnull().all(), "Coluna 'label' possui valores nulos."
assert df["text"].apply(lambda x: isinstance(x, str)).all(), "Coluna 'text' deve conter apenas strings."
assert df["label"].apply(lambda x: x in ["positivo", "negativo"]).all(), "Coluna 'label' deve conter apenas 'positivo' ou 'negativo'."
print("Validações concluídas com sucesso!")

Validações concluídas com sucesso!


Separacao: Treino e teste

In [113]:
x_train, x_test, y_train, y_test = train_test_split(df["text"], df["label"], test_size=0.2, random_state=42)

Pipeline: exibicao

In [114]:
pipeline = Pipeline([
    ("vect", CountVectorizer()),
    ("clf", MultinomialNB())])

pipeline.fit(x_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', MultinomialNB())])

Avaliar modelo

In [115]:
y_pred = pipeline.predict(x_test)
print(classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

    negativo       1.00      0.67      0.80         3
    positivo       0.50      1.00      0.67         1

    accuracy                           0.75         4
   macro avg       0.75      0.83      0.73         4
weighted avg       0.88      0.75      0.77         4



Salvar o modelo e vetor

In [116]:
joblib.dump(pipeline.named_steps["clf"], "../model.joblib")
joblib.dump(pipeline.named_steps["vect"], "../vectorizer.joblib")

['../vectorizer.joblib']

Rastrear os experimentos com MLFLOW

In [117]:
mlflow.set_experiment("sentimento_pipeline")
with mlflow.start_run():
    mlflow.log_param("modelo", "MultinomialNB")
    mlflow.log_param("vetor", "CountVectorizer")
    mlflow.sklearn.log_model(pipeline, "modelo_sentimento")

2025/07/18 20:25:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/18 20:25:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
